In [1]:
%pip install langchain langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
# loading the text file conversation
from langchain.document_loaders import TextLoader

# use os module to go through the directory and load the text files
import os 
# Store all loaded documents
documents = []

data_dir = "data"  # replace with your directory
# Loop through all files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_dir, filename)
        loader = TextLoader(file_path)
        documents.extend(loader.load())  # append loaded documents


In [3]:
# split the big text into smaller chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [4]:
# embedding the text chunks

from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_14819/2721590312.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding=embedding_model)
retriever = vectorstore.as_retriever()


In [6]:
from langchain.llms import Ollama

# update this to load the different model

# Initialize Ollama with the gemma3 model
llm = Ollama(model="gemma3")
  

/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_14819/1557086837.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3")


In [7]:
from langchain.chains import RetrievalQA
# RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Ask a question
query = "What is the summary of the text?"
response = rag_chain(query)

print("Answer:", response['result'])

/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_14819/2052742362.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain(query)


Answer: The text summarizes the life and career of Robert Frost, detailing his early struggles with publication in America, his eventual success in England, and his rise to becoming a renowned and celebrated poet, including his four Pulitzer Prizes and the publication of his iconic New England poems.


In [ ]:
# maintain chat history as well

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOllama

# Create LLM
llm = ChatOllama(model="gemma3")  # Or another Ollama model you're using

# Set output_key INSIDE memory (this is the critical fix)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  
)

# Create the RAG chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

# Run query
response1 = rag_chain({"question": "What is the summary of the text?"})
print("Answer:", response1["answer"])

response2 = rag_chain({"question": "What is his most famous poem?"})
print("Answer:", response2["answer"])


Answer: Here’s a summary of the text:

The text details the life and career of Robert Frost. It highlights his early struggles with publication in America, his eventual success in England, and his remarkable achievements, including four Pulitzer Prizes and becoming a famous poet. Key events include the naming of a mountain in his honor, his role at John F. Kennedy’s inauguration, and the publication of his influential books, *North of Boston* and *A Boy’s Will*.
Answer: The text doesn't state which of Robert Frost’s poems is his “most famous.” It mentions several poems he wrote, including “My Butterfly” and “Dedication,” but doesn’t rank them.
